In [5]:
import os
import glob
import torch
import torchvision.transforms.functional as TF
import numpy as np
import cv2

from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

# 데이터 불러오기 및 데이터 전처리

## 이미지 리사이즈

In [2]:
#이미지 경로
fake_image_path = "Dataset\fake"
nfake_image_path = "Dataset\nfake"

In [3]:
def resize_image(input_path, output_path, new_size):
    """
    이미지를 불러와서 새로운 크기로 리사이즈하는 함수
    :param input_path: 원본 이미지 파일 경로
    :param output_path: 리사이즈된 이미지를 저장할 파일 경로
    :param new_size: 새로운 크기 (너비, 높이) 튜플 형태로 전달
    """
    try:
        # 이미지 열기
        with Image.open(input_path) as img:
            # 리사이즈
            resized_img = img.resize(new_size)
            # 리사이즈된 이미지 저장
            resized_img.save(output_path)
    except Exception as e:
        print(f"오류 발생: {e}")

In [4]:
def resize_images_in_directory(directory, output_directory, new_size):
    """
    디렉토리 내에 있는 이미지들을 리사이즈하는 함수
    :param directory: 원본 이미지 파일들이 있는 디렉토리 경로
    :param output_directory: 리사이즈된 이미지를 저장할 디렉토리 경로
    :param new_size: 새로운 크기 (너비, 높이) 튜플 형태로 전달
    """
    # 디렉토리 내의 모든 이미지 파일들을 가져옴
    image_files = glob.glob(os.path.join(directory, "*.jpg")) + glob.glob(os.path.join(directory, "*.jpeg")) + glob.glob(os.path.join(directory, "*.png"))

    # 출력 디렉토리가 없으면 생성
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # 이미지들을 순회하면서 리사이즈 수행
    for image_file in image_files:
        filename = os.path.basename(image_file)
        output_path = os.path.join(output_directory, filename)
        resize_image(image_file, output_path, new_size)

In [5]:
# 사용 예시
input_directory = r"Dataset\fake"
output_directory = r"Dataset\r_fake"
new_size = (224, 224)  # 새로운 크기 (너비, 높이)
resize_images_in_directory(input_directory, output_directory, new_size)

In [6]:
input_directory = r"Dataset\nfake"
output_directory = r"Dataset\r_nfake"
new_size = (224, 224)  # 새로운 크기 (너비, 높이)
resize_images_in_directory(input_directory, output_directory, new_size)

## 이미지 라벨링

In [7]:
def adjust_hls(hls_image, lightness_scale=1.0, saturation_scale=1.0):
    h, l, s = cv2.split(hls_image)  # HLS 이미지를 각 채널로 분리
    l = np.clip(l * lightness_scale, 0, 255).astype(np.uint8)  # 밝기 조절
    s = np.clip(s * saturation_scale, 0, 255).astype(np.uint8)  # 채도 조절
    adjusted_hls = cv2.merge([h, l, s])  # 조정된 채널을 다시 합침
    return adjusted_hls

In [8]:
def labeling(folder_paths):
    images = []
    labels = []
    for folder_path in folder_paths:
        if not os.path.exists(folder_path):
            print(f"경로가 존재하지 않습니다: {folder_path}")
            continue
        label = 1 if 'r_fake' in folder_path.lower() else 0
        for filename in os.listdir(folder_path):
            image_path = os.path.join(folder_path, filename)
            if image_path.endswith('.jpg') or image_path.endswith('.png'):
                image = Image.open(image_path).convert('RGB')
                image_array = np.array(image)[:, :, ::-1]  # RGB to BGR
                hls_image = cv2.cvtColor(image_array, cv2.COLOR_BGR2HLS)
                adjusted_hls_image = adjust_hls(hls_image, lightness_scale=1.2, saturation_scale=0.9)
                # 이미지 정규화 (픽셀 값의 범위를 0 ~ 1 사이로 조정)
                normalized_image = adjusted_hls_image / 255.0
                images.append(normalized_image)
                labels.append(label)
    return images, labels

In [5]:
# 스크립트가 PS3 폴더 내에 있으므로 상대 경로를 사용합니다.
folder_paths = ["Dataset/r_fake", "Dataset/r_nfake"]
images, labels = labeling(folder_paths)

# 결과 확인
print("이미지 개수:", len(images))
print("라벨 개수:", len(labels))

이미지 개수: 12229
라벨 개수: 12229


In [10]:
class CustomDataset(Dataset):
    def __init__(self, folder_paths, transform=None):
        self.images, self.labels = labeling(folder_paths)
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        
        if self.transform:
            # numpy 이미지를 PIL 이미지로 변환합니다. 이 작업이 필요한 이유는 torchvision의 transforms는 PIL 이미지를 기대하기 때문입니다.
            image = Image.fromarray((image * 255).astype(np.uint8))
            image = self.transform(image)
        
        return image, label

In [9]:
# 데이터셋을 사용하기 위한 transform 정의 , RESNET-50(224*224를 입력으로 받음)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 이미지 크기 조정
    transforms.ToTensor(),  # PIL 이미지를 PyTorch Tensor로 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # 이미지 정규화
])

In [8]:
import pickle

# 데이터셋을 파일로 저장하는 함수
def save_dataset(dataset, filename):
    with open(filename, 'wb') as f:
        pickle.dump(dataset, f)

# 파일에서 데이터셋을 로드하는 함수
def load_dataset(filename):
    with open(filename, 'rb') as f:
        dataset = pickle.load(f)
    return dataset

In [ ]:
# 전체 데이터셋 로드
full_dataset = CustomDataset(["Dataset/r_fake", "Dataset/r_nfake"], transform=transform)
# 데이터셋을 파일로 저장
# 저장할 파일명 설정
filename = "dataset.pkl"
save_dataset(full_dataset, filename)

In [14]:
# 저장된 파일을 로드하여 데이터셋 사용
loaded_dataset = load_dataset(filename)

In [13]:
full_dataset = CustomDataset(["Dataset/r_fake", "Dataset/r_nfake"], transform=transform)

In [7]:
from torch.utils.data import random_split

# 데이터셋 로드
dataset = full_dataset

# 데이터셋 분할
train_size = int(len(dataset) * 0.7)
test_size = int(len(dataset) * 0.15)
val_size = len(dataset) - train_size - test_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])


In [11]:
# DataLoader 설정
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

NameError: name 'train_dataset' is not defined

# 모델

In [17]:
import torch
import numpy as np
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from transformers import ViTForImageClassification, ViTFeatureExtractor
from PIL import Image

In [18]:
def initialize_model():
    model_id = "google/vit-base-patch16-224"
    model = ViTForImageClassification.from_pretrained(model_id)

    # 이진 분류를 위해 출력 레이어 수정
    model.classifier = torch.nn.Linear(model.config.hidden_size, 2)
    
    feature_extractor = ViTFeatureExtractor.from_pretrained(model_id)
    
    return model, feature_extractor

model, feature_extractor = initialize_model()

c:\Users\kwonh\anaconda3\envs\AllLeave\Lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [16]:
import torch.nn as nn
# 모델 학습 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()
num_epochs = 100

In [17]:
import tqdm as tqdm
from torch.utils.tensorboard import SummaryWriter

In [18]:
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

# 얼리 스톱핑을 위한 초기 설정
patience = 2  # 성능 향상이 없는 경우, 몇 에폭 동안 기다릴 것인지
val_loss_min = np.Inf  # 가능한 무한대 값으로 초기화
patience_counter = 0  # 현재 기다리고 있는 에폭 수
writer = SummaryWriter()

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_corrects = 0
    
    for inputs, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = loss_fn(outputs.logits, labels)  # outputs.logits 사용
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * inputs.size(0)
        preds = torch.sigmoid(outputs.logits) >= 0.5  # outputs.logits 사용
        train_corrects += (preds == labels.unsqueeze(1)).float().sum()  # 수정된 정확도 계산

    train_loss = train_loss / len(train_loader.dataset)
    train_acc = train_corrects / len(train_loader.dataset)  # double 대신 바로 계산
    
    model.eval()
    val_loss = 0.0
    val_corrects = 0
    
    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc="Validating"):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            loss = loss_fn(outputs.logits, labels)  # outputs.logits 사용
            
            val_loss += loss.item() * inputs.size(0)
            preds = torch.sigmoid(outputs.logits) >= 0.5  # outputs.logits 사용
            val_corrects += (preds == labels.unsqueeze(1)).float().sum()  # 수정된 정확도 계산

    val_loss = val_loss / len(val_loader.dataset)
    val_acc = val_corrects / len(val_loader.dataset)
    
    writer.add_scalar("Loss/train", train_loss, epoch)
    writer.add_scalar("Loss/val", val_loss, epoch)
    writer.add_scalar("Accuracy/train", train_acc, epoch)
    writer.add_scalar("Accuracy/val", val_acc, epoch)
    
    print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Train Acc={train_acc:.4f}, Val Loss={val_loss:.4f}, Val Acc={val_acc:.4f}")
    
    # 얼리 스톱핑 조건 검사
    if val_loss < val_loss_min:
        print(f"Validation loss decreased ({val_loss_min:.6f} --> {val_loss:.6f}). Saving model ...")
        torch.save(model.state_dict(), "ViT_classification_model.pth")
        val_loss_min = val_loss
        patience_counter = 0  # 리셋
    else:
        patience_counter += 1
        print(f"EarlyStopping counter: {patience_counter} out of {patience}")
        if patience_counter >= patience:
            print("Early stopping initiated. Training stopped.")
            break

writer.close()


Training Epoch 1:   0%|          | 0/268 [00:00<?, ?it/s]

Validating: 100%|██████████| 58/58 [02:00<00:00,  2.07s/it]


Epoch 1: Train Loss=0.0867, Train Acc=0.9996, Val Loss=0.0554, Val Acc=0.9967
Validation loss decreased (inf --> 0.055354). Saving model ...


Validating: 100%|██████████| 58/58 [03:27<00:00,  3.59s/it]


Epoch 2: Train Loss=0.0842, Train Acc=1.0001, Val Loss=0.0596, Val Acc=1.0000
EarlyStopping counter: 1 out of 2


Validating: 100%|██████████| 58/58 [03:28<00:00,  3.59s/it]

Epoch 3: Train Loss=0.1528, Train Acc=0.9994, Val Loss=0.1012, Val Acc=1.0000
EarlyStopping counter: 2 out of 2
Early stopping initiated. Training stopped.


In [3]:
import torch
import torchvision.models as models
import torch
from torchvision import models
from PIL import Image
import numpy as np

In [29]:
image_path = r"C:\Users\kwonh\Desktop\ps3\Dataset\test\3.jpg"


image = Image.open(image_path).convert('RGB')
image_array = np.array(image)[:, :, ::-1]  # RGB to BGR
hls_image = cv2.cvtColor(image_array, cv2.COLOR_BGR2HLS)
adjusted_hls_image = adjust_hls(hls_image, lightness_scale=1.2, saturation_scale=0.9)
# 이미지 정규화 (픽셀 값의 범위를 0 ~ 1 사이로 조정)
normalized_image = adjusted_hls_image / 255.0
image = Image.fromarray((normalized_image * 255).astype(np.uint8))
image = transform(image)

In [32]:
# 모델 상태 로드
from tqdm import tqdm
model.load_state_dict(torch.load("ViT_classification_model.pth"))

input_tensor = image.unsqueeze(0)  # 배치 차원 추가
# 모델을 평가 모드로 설정 (예측할 때는 evaluation 모드로 설정)
model.eval()

with torch.no_grad():
    
    inputs = input_tensor
            
    outputs = model(inputs)
    preds = torch.sigmoid(outputs.logits) >= 0.5
            

# 결과 출력 (여러 요소에 대한 처리)
for idx, pred in enumerate(preds.squeeze()):  # squeeze()를 사용하여 불필요한 차원 제거
    if pred:
        print(f"이미지 {idx+1}: 합성입니다.")
    else:
        print(f"이미지 {idx+1}: 합성이 아닙니다.")





이미지 1: 합성입니다.
이미지 2: 합성이 아닙니다.
